In [1]:
from os import cpu_count

from collections import OrderedDict
from typing import List, Tuple, Dict

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10
from torchinfo import summary

import flwr as fl
from flwr.common import Metrics

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Training on cuda using PyTorch 2.2.1+cu121 and Flower 1.7.0


## Use A Federated Learning Strategy

In [2]:
# hyper-parameters

NUM_CLIENT = 10
EPOCHS_CLIENT = 1
BATCH_SIZE = 32
NUM_WORKER = cpu_count()

### Load Torchvision CIFAR10 Dataset

In [3]:
### Loading The Dataset And Partitioning ###

def load_datasets(num_clients: int):
    
    # image transformation
    img_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])
    
    # loading the torchvision dataset
    train_dataset = CIFAR10(
        root='./dataset',
        train=True,
        transform=img_transform,
        download=True
    )

    test_dataset = CIFAR10(
        root='./dataset',
        train=False,
        transform=img_transform,
        download=True
    )

    # split training dataset into partitions
    partition_size = len(train_dataset) // num_clients
    lengths = [partition_size] * num_clients
    train_part_dataset = random_split(
        dataset=train_dataset,
        lengths=lengths,
        generator=torch.Generator().manual_seed(42)
        )
    
    # create all client train and val dataloaders
    train_dataloaders = []
    val_dataloaders = []

    # split partition into train and val datasets and wrap into torch dataloaders
    for dataset in train_part_dataset:
        # split the partition
        split_dataset = random_split(
            dataset=dataset,
            lengths=[0.8, 0.2], # train & val dataset split fraction
            generator=torch.Generator().manual_seed(42)
        )
        # wrap with torch dataloader and add to dataloader list
        partition_train_dl = DataLoader(
            dataset=split_dataset[0],
            batch_size=BATCH_SIZE,
            shuffle=True,
            num_workers=NUM_WORKER
        )
        partition_val_dl = DataLoader(
            dataset=split_dataset[1],
            batch_size=BATCH_SIZE,
            num_workers=NUM_WORKER
        )
        train_dataloaders.append(partition_train_dl)
        val_dataloaders.append(partition_val_dl)
        
    # create test dataloader from the test split (Dataset) with transform function
    test_dataloader = DataLoader(
        dataset=test_dataset,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKER
    )

    # return all the train (partitioned), val (partitioned) & test dataloaders
    return train_dataloaders, val_dataloaders, test_dataloader

In [4]:
train_dataloaders, val_dataloaders, test_dataloader = load_datasets(NUM_CLIENT)

100%|██████████| 170498071/170498071 [00:23<00:00, 7341435.46it/s] 


Extracting ./dataset/cifar-10-python.tar.gz to ./dataset
Files already downloaded and verified


### Model Training & Eval Client Code

In [5]:
### Defining The Model ###

class TinyVGG(nn.Module):
    """Creates the TinyVGG architecture for 32*32 Image Data"""
        
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=10, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features= 10 * 8 * 8, out_features=10)
        )

    def forward(self, x: torch.tensor) -> torch.tensor:
        return self.classifier(self.block2(self.block1(x)))

### Train and Test Function For FLOWER Clients
    
# calculate accuracy
def accuracy_fn(y_pred: torch.tensor, y_true: torch.tensor) -> float:
    """Calculates the accuracy of a model on given predictions

    Args:
        y_pred: predicted labels
        y_true: true labels
    
    Returns:
        A float value which is the calculated accuracy.
    """
    return ((torch.eq(y_pred, y_true).sum().item() / len(y_true)) * 100)

# fit the model on training data
def train(model: torch.nn.Module,
          data_loader: torch.utils.data.DataLoader,
          epochs: int,
          device: torch.device,
          verbose=False,
          loss_fn: torch.nn.Module = None,
          optimizer: torch.optim.Optimizer = None) -> Tuple[float, float]:
    """Trains a PyTorch model for the given epochs.

    Turns a target PyTorch model to training mode and then
    runs through all of the required training steps (forward
    pass, loss calculation, optimizer step).

    Args:
        model: A PyTorch model to be trained.
        dataloader: A DataLoader instance for the model to be trained on.
        epochs: Epochs.
        device: A target device to compute on (e.g. "cuda" or "cpu").
        verbose: A boolean value to see the model metrics (loss and accuracy)
        loss_fn: A PyTorch loss function to minimize.
        optimizer: A PyTorch optimizer to help minimize the loss function.
    
    Returns:
        A tuple of training loss and training accuracy metrics.
        In the form (train_loss, train_accuracy). For example:

        (0.0223, 0.8985)
    """

    # optimizer and criterion (loss_fn) if None given
    if loss_fn == None:
        loss_fn = torch.nn.CrossEntropyLoss()
    if optimizer == None:
        optimizer = torch.optim.SGD(model.parameters())

    model.train() # model in train mode
    total_epoch_loss, total_epoch_acc = 0, 0
    for epoch in range(epochs):
        
        train_loss, train_acc = 0, 0
        
        for X, y in data_loader:
            # get data to device
            X = X.to(device)
            y = y.to(device)

            # forward pass
            y_logit = model(X)
            loss = loss_fn(y_logit, y)        

            # backward pass
            optimizer.zero_grad() # empty param's grad
            loss.backward() # backward propagation
            optimizer.step() # updata params (take the gradient descent step)

            # Metrics
            # calculate loss and accuracy per batch
            train_loss += loss.item() * len(y)
            y_pred_labels = torch.argmax(y_logit, dim=1)
            train_acc += accuracy_fn(y_pred_labels, y)
        
        # per epoch
        train_loss /= len(data_loader.dataset)
        train_acc /= len(data_loader)
        
        if verbose:
            print(f"Epoch {epoch+1} | Train Loss {train_loss:.4f} | Train Acc {train_acc:.2f}")

        # for all epochs
        total_epoch_loss += train_loss
        total_epoch_acc += train_acc
    
    return (total_epoch_loss / epochs, total_epoch_acc / epochs)
    

# test the model on test data
def test(model: torch.nn.Module,
         data_loader: torch.utils.data.DataLoader,
         device: torch.device,
         loss_fn: torch.nn.Module=None) -> Tuple[float, float]:
    """Tests a PyTorch model for the given epochs.

    Turns a target PyTorch model to "eval" mode and then performs
    a forward pass on a testing dataset.

    Args:
        model: A PyTorch model to be tested.
        data_loader: A DataLoader instance for the model to be tested on.
        device: A target device to compute on (e.g. "cuda" or "cpu").
        loss_fn: A PyTorch loss function to calculate loss on the test data.

    Returns:
        A tuple of testing loss and testing accuracy metrics.
        In the form (test_loss, test_accuracy). For example:

        (0.0223, 0.8985)
    """

    test_loss, test_acc = 0, 0

    # criterion (loss_fn) if None given
    if loss_fn == None:
        loss_fn = torch.nn.CrossEntropyLoss()

    model.eval() # model in evaluation mode
    with torch.inference_mode():
        for X, y in data_loader:
            # get data to device
            X = X.to(device)
            y = y.to(device)
            
            # forward pss
            y_logit = model(X)
            loss = loss_fn(y_logit, y)

            # calculate loss and accuracy per batch
            test_loss += loss.item() * len(y)
            y_pred_labels = torch.argmax(y_logit, dim=1)
            test_acc += accuracy_fn(y_pred_labels, y)

    test_loss /= len(data_loader.dataset)
    test_acc /= len(data_loader)
    return (test_loss, test_acc)

### Updating Model Parameters (helper functions from client's perspective)

# set client paramters
def set_parameters(model: nn.Module, parameters: List[np.ndarray]):
    # de-serialize the ndarray to tensors
    parameters = [torch.from_numpy(np_arr).to(dtype=torch.float32, device=DEVICE) for np_arr in parameters]
    # match every weight with its model block
    param_dict = zip(model.state_dict().keys(), parameters)
    # convert the param_dict to ordered dict and load back into the model
    model.load_state_dict(OrderedDict(param_dict), strict=True)

# get parameters from client
def get_parameters(model: nn.Module) -> List[np.ndarray]:
    # serialize the model weights into ndarray and return
    return [weights.cpu().numpy() for _, weights in model.state_dict().items()]

### Customizing FLOWER Client

In [6]:
### Implementing A FLOWER Client using NumPyClient

class CustomClient(fl.client.NumPyClient):
    """A custom client implementation representing an organization with model and data"""

    # pass a model, train and test dataloader
    def __init__(self, client_id: int, model, train_dataloader, val_dataloader) -> None:
        super().__init__()
        self.cid = client_id
        self.model = model
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader

    # return the current local model parameters to server
    def get_parameters(self, config):
        print(f'[Client {self.cid}] get_parameters')
        return get_parameters(self.model)
    
    # receive global model parameters, train and return the updated parameters with other metrics/arbitrary values
    def fit(self, parameters, config):
        print(f'[Client {self.cid}] fit, config: {config}')
        set_parameters(self.model, parameters)
        loss, accuracy = train(model=self.model, data_loader=self.train_dataloader, epochs=EPOCHS_CLIENT, device=DEVICE)
        return get_parameters(self.model), len(self.train_dataloader), {'loss': loss, 'accuracy': accuracy} # updated parameters, num_batches, metrics
    
    # receive global model parameters, evaluate and return the metrics/arbitrary values
    def evaluate(self, parameters, config) -> Tuple[float | int | Dict[str, bool | bytes | float | int | str]]:
        print(f'[Client {self.cid}] evaluate, config: {config}')
        set_parameters(self.model, parameters)
        loss, accuracy = test(model=self.model, data_loader=self.val_dataloader, device=DEVICE)
        return float(loss), len(self.val_dataloader), {'accuracy': accuracy} # loss, num_batches, metrics
    
### client_fun: To create a CustomClient instance on demand when requested from FLOWER Framework

def client_fn(client_id: str) -> CustomClient:
    """Create a FLOWER client representing a single organization

    Creates an instance of CustomClient based on the client_id provided
    to load client specific data partition.

    Args:
        client_id: An str to load a specific client.

    Returns:
        A CustomClient (fl.client.NumPyClient) instance representing a FLOWER client.
    """

    # create an instance of the model
    model = TinyVGG().to(DEVICE)

    # load the client specific dataloaders
    train_dl = train_dataloaders[int(client_id)]
    val_dl = val_dataloaders[int(client_id)]

    # create client instance, convert object to Client type and return
    return CustomClient(client_id=int(client_id), model=model, train_dataloader=train_dl, val_dataloader=val_dl).to_client()

In [7]:
### client metric aggregation functions

# aggregate the metrics received from all the client's evaluate function
def eval_weighted_avg(metrics: List[Tuple[int, Metrics]]):
    #  multiply accuracy with each client's number of samples / or is it batch size??
    accuracies = [num_examples * m['accuracy'] for num_examples, m in metrics]
    num_samples = [num_examples for num_examples, _ in metrics]

    # aggregate and return the custom metrics (weighted avg)
    return {'test_accuracy': sum(accuracies) / sum(num_samples)}

# aggregate the metrics received from all the client's fit function
def fit_weighted_avg(metrics: List[Tuple[int, Metrics]]):
    #  multiply accuracy with each client's number of samples / or is it batch size??
    accuracies = [num_examples * m['accuracy'] for num_examples, m in metrics]
    num_samples = [num_examples for num_examples, _ in metrics]
    # for loss
    losses = [num_examples * m['loss'] for num_examples, m in metrics]

    # aggregate and return the custom metrics (weighted avg)
    return {'train_loss': sum(losses) / sum(num_samples), 'train_accuracy': sum(accuracies) / sum(num_samples)}

### Customizations

1. Server-side parameter initialization: initialized model params at server
2. Changing strategy: FedAvg, FedAdagrad etc
3. Server-side evaluation: test/val at server
4. Sending/receiving arbitrary values to/from clients: configure/set client-side params from server side

In [8]:
### 1. server-side initialization
## get serialized (numpy version) parameters of the model
params = get_parameters(TinyVGG())
## pass the parameters to the strategy
# strategy = fl.server.strategy.FedAvg(
#     fraction_fit=1.0, # C: fraction of client to choose for training
#     fraction_evaluate=0.5, # fraction of client to choose for evaluation
#     min_fit_clients=10, # minimum clients needed for training
#     min_evaluate_clients=5, # minimum clients needed for evaluation
#     min_available_clients=10, # wait till given client are available
#     evaluate_metrics_aggregation_fn=eval_weighted_avg, # aggregate the val metrics of clients
#     fit_metrics_aggregation_fn=fit_weighted_avg, # aggregate the train metrics of clients
#     initial_parameters=fl.common.ndarrays_to_parameters(params), # <-- passed parameters
# )

### 2. changing strategy to FedAdagrad
# strategy = fl.server.strategy.FedAdagrad(
#     fraction_fit=1.0, # C: fraction of client to choose for training
#     fraction_evaluate=0.5, # fraction of client to choose for evaluation
#     min_fit_clients=10, # minimum clients needed for training
#     min_evaluate_clients=5, # minimum clients needed for evaluation
#     min_available_clients=10, # wait till given client are available
#     evaluate_metrics_aggregation_fn=eval_weighted_avg, # aggregate the metrics of clients
#     fit_metrics_aggregation_fn=fit_weighted_avg, # aggregate the metrics of clients
#     initial_parameters=fl.common.ndarrays_to_parameters(params), # initial parameters
# )

### 3. Server-side parameter evaluation
def eval_server(server_round: int, params: fl.common.NDArray, config: Dict[str, fl.common.Scalar]):
    # create server model
    model = TinyVGG().to(DEVICE)
    # load validation dataloader
    val_dl = val_dataloaders[0]
    # update model with latest parameters
    set_parameters(model=model, parameters=params)
    # perform centralized evaluation
    loss, accuracy = test(model=model, data_loader=val_dl, device=DEVICE)
    print(f'Server round {server_round}: evaluation loss {loss} | accuracy {accuracy}')
    return loss, {'accuracy': accuracy}

# strategy = fl.server.strategy.FedAvg(
#     fraction_fit=1.0, # C: fraction of client to choose for training
#     fraction_evaluate=0.5, # fraction of client to choose for evaluation
#     min_fit_clients=10, # minimum clients needed for training
#     min_evaluate_clients=5, # minimum clients needed for evaluation
#     min_available_clients=10, # wait till given client are available
#     # fit_metrics_aggregation_fn=fit_weighted_avg, # aggregate the metrics of clients
#     initial_parameters=fl.common.ndarrays_to_parameters(params), # init parameters passed
#     evaluate_fn=eval_server, # <-- server evaluation function passed here
# )

### 4: Sending/receiving arbitrary values to/from clients: configure/set client-side params from server side

# FLOWER Client with client specific epochs send from server through config parameter in fit and eval functions
class CustomClient(fl.client.NumPyClient):
    """A custom client implementation representing an organization with model and data"""

    # pass a model, train and test dataloader
    def __init__(self, client_id: int, model, train_dataloader, val_dataloader) -> None:
        super().__init__()
        self.cid = client_id
        self.model = model
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader

    # return the current local model parameters to server
    def get_parameters(self, config):
        print(f'[Client {self.cid}] get_parameters')
        return get_parameters(self.model)
    
    # receive global model parameters, train and return the updated parameters with other metrics
    def fit(self, parameters, config):
        # get client specific epochs from config
        server_round = config['server_round']
        client_epochs = config['local_epochs']

        # use the values provided by config
        print(f'[Client {self.cid}] round {server_round} fit, config: {config}')
        set_parameters(self.model, parameters)
        loss, accuracy = train(model=self.model, data_loader=self.train_dataloader, epochs=client_epochs, device=DEVICE)
        return get_parameters(self.model), len(self.train_dataloader), {'loss': loss, 'accuracy': accuracy} # updated parameters, num_batches, metrics
    
    # receive global model parameters, evaluate and return the metrics
    def evaluate(self, parameters, config) -> Tuple[float | int | Dict[str, bool | bytes | float | int | str]]:
        print(f'[Client {self.cid}] evaluate, config: {config}')
        set_parameters(self.model, parameters)
        loss, accuracy = test(model=self.model, data_loader=self.val_dataloader, device=DEVICE)
        return float(loss), len(self.val_dataloader), {'accuracy': accuracy} # loss, num_batches, metrics

# training configuration from server to client
def fit_config(server_round: int):
    """Return training configuration dict for each round.

    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        'server_round': server_round,
        'local_epochs': 1 if server_round < 2 else 2
    }
    return config

strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0, # C: fraction of client to choose for training
    fraction_evaluate=0.5, # fraction of client to choose for evaluation
    min_fit_clients=10, # minimum clients needed for training
    min_evaluate_clients=5, # minimum clients needed for evaluation
    min_available_clients=10, # wait till given client are available
    # evaluate_metrics_aggregation_fn=eval_weighted_avg, # aggregate the val metrics of clients
    # fit_metrics_aggregation_fn=fit_weighted_avg, # aggregate the train metrics of clients
    initial_parameters=fl.common.ndarrays_to_parameters(params), # init parameters passed
    evaluate_fn=eval_server, # server evaluation function passed here
    on_fit_config_fn=fit_config, # <-- client fit config send from server/strategy
)


In [9]:
### Start The Training

# client resources (allocate cpus and gpus)
client_resources = {'num_cpus': NUM_WORKER//NUM_CLIENT, 'num_gpus': 1 if torch.cuda.is_available() else 0}

# start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENT,
    config=fl.server.ServerConfig(num_rounds=3),
    strategy=strategy,
    client_resources=client_resources
)

INFO flwr 2024-04-02 15:34:13,429 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)


2024-04-02 15:34:15,084	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-04-02 15:34:15,862 | app.py:213 | Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'accelerator_type:G': 1.0, 'GPU': 1.0, 'object_store_memory': 6315348787.0, 'node:10.255.93.233': 1.0, 'memory': 12630697575.0, 'CPU': 16.0}
INFO flwr 2024-04-02 15:34:15,863 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2024-04-02 15:34:15,863 | app.py:242 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 1}
INFO flwr 2024-04-02 15:34:15,869 | app.py:288 | Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO flwr 2024-04-02 15:34:15,869 | server.py:89 | Initializing global parameters
INFO flwr 2024-04-02 15:34:15,869 | server.py:272 | Using initial parameters provided by strategy
INFO flwr 2024-04-02 15:34:15,870 | server.py:91 | Evaluating initial parameters
INF

Server round 0: evaluation loss 2.3042768020629882 | accuracy 8.3984375
(DefaultActor pid=1039175) [Client 7] round 1 fit, config: {'server_round': 1, 'local_epochs': 1}
(DefaultActor pid=1039175) [Client 6] round 1 fit, config: {'server_round': 1, 'local_epochs': 1}
(DefaultActor pid=1039175) [Client 5] round 1 fit, config: {'server_round': 1, 'local_epochs': 1}
(DefaultActor pid=1039175) [Client 8] round 1 fit, config: {'server_round': 1, 'local_epochs': 1}
(DefaultActor pid=1039175) [Client 9] round 1 fit, config: {'server_round': 1, 'local_epochs': 1}
(DefaultActor pid=1039175) [Client 1] round 1 fit, config: {'server_round': 1, 'local_epochs': 1}
(DefaultActor pid=1039175) [Client 0] round 1 fit, config: {'server_round': 1, 'local_epochs': 1}
(DefaultActor pid=1039175) [Client 3] round 1 fit, config: {'server_round': 1, 'local_epochs': 1}
(DefaultActor pid=1039175) [Client 4] round 1 fit, config: {'server_round': 1, 'local_epochs': 1}
(DefaultActor pid=1039175) [Client 2] round 1 

DEBUG flwr 2024-04-02 15:34:30,794 | server.py:236 | fit_round 1 received 10 results and 0 failures
WARNING flwr 2024-04-02 15:34:30,799 | fedavg.py:250 | No fit_metrics_aggregation_fn provided
INFO flwr 2024-04-02 15:34:31,171 | server.py:125 | fit progress: (1, 2.3041712799072265, {'accuracy': 8.3984375}, 14.68658795603551)
DEBUG flwr 2024-04-02 15:34:31,171 | server.py:173 | evaluate_round 1: strategy sampled 5 clients (out of 10)


Server round 1: evaluation loss 2.3041712799072265 | accuracy 8.3984375
(DefaultActor pid=1039175) [Client 6] evaluate, config: {}
(DefaultActor pid=1039175) [Client 9] evaluate, config: {}
(DefaultActor pid=1039175) [Client 0] evaluate, config: {}
(DefaultActor pid=1039175) [Client 5] evaluate, config: {}
(DefaultActor pid=1039175) [Client 2] evaluate, config: {}


...

(DefaultActor pid=1039175) [Client 4] round 3 fit, config: {'server_round': 3, 'local_epochs': 2}
(DefaultActor pid=1039175) [Client 9] round 3 fit, config: {'server_round': 3, 'local_epochs': 2}
(DefaultActor pid=1039175) [Client 5] round 3 fit, config: {'server_round': 3, 'local_epochs': 2}
(DefaultActor pid=1039175) [Client 2] round 3 fit, config: {'server_round': 3, 'local_epochs': 2}
(DefaultActor pid=1039175) [Client 8] round 3 fit, config: {'server_round': 3, 'local_epochs': 2}
(DefaultActor pid=1039175) [Client 3] round 3 fit, config: {'server_round': 3, 'local_epochs': 2}
(DefaultActor pid=1039175) [Client 1] round 3 fit, config: {'server_round': 3, 'local_epochs': 2}
(DefaultActor pid=1039175) [Client 7] round 3 fit, config: {'server_round': 3, 'local_epochs': 2}
(DefaultActor pid=1039175) [Client 6] round 3 fit, config: {'server_round': 3, 'local_epochs': 2}
(DefaultActor pid=1039175) [Client 0] round 3 fit, config: {'server_round': 3, 'local_epochs': 2}


DEBUG flwr 2024-04-02 15:35:24,368 | server.py:236 | fit_round 3 received 10 results and 0 failures
INFO flwr 2024-04-02 15:35:24,735 | server.py:125 | fit progress: (3, 2.303821851730347, {'accuracy': 8.3984375}, 68.2505871958565)
DEBUG flwr 2024-04-02 15:35:24,735 | server.py:173 | evaluate_round 3: strategy sampled 5 clients (out of 10)


Server round 3: evaluation loss 2.303821851730347 | accuracy 8.3984375
(DefaultActor pid=1039175) [Client 2] evaluate, config: {}
(DefaultActor pid=1039175) [Client 3] evaluate, config: {}
(DefaultActor pid=1039175) [Client 7] evaluate, config: {}
(DefaultActor pid=1039175) [Client 1] evaluate, config: {}
(DefaultActor pid=1039175) [Client 4] evaluate, config: {}


DEBUG flwr 2024-04-02 15:35:30,798 | server.py:187 | evaluate_round 3 received 5 results and 0 failures
INFO flwr 2024-04-02 15:35:30,798 | server.py:153 | FL finished in 74.313835040899
INFO flwr 2024-04-02 15:35:30,799 | app.py:226 | app_fit: losses_distributed [(1, 2.304241261291504), (2, 2.3035378883361814), (3, 2.303084147262573)]
INFO flwr 2024-04-02 15:35:30,799 | app.py:227 | app_fit: metrics_distributed_fit {}
INFO flwr 2024-04-02 15:35:30,799 | app.py:228 | app_fit: metrics_distributed {}
INFO flwr 2024-04-02 15:35:30,799 | app.py:229 | app_fit: losses_centralized [(0, 2.3042768020629882), (1, 2.3041712799072265), (2, 2.303984405517578), (3, 2.303821851730347)]
INFO flwr 2024-04-02 15:35:30,800 | app.py:230 | app_fit: metrics_centralized {'accuracy': [(0, 8.3984375), (1, 8.3984375), (2, 8.3984375), (3, 8.3984375)]}


History (loss, distributed):
	round 1: 2.304241261291504
	round 2: 2.3035378883361814
	round 3: 2.303084147262573
History (loss, centralized):
	round 0: 2.3042768020629882
	round 1: 2.3041712799072265
	round 2: 2.303984405517578
	round 3: 2.303821851730347
History (metrics, centralized):
{'accuracy': [(0, 8.3984375), (1, 8.3984375), (2, 8.3984375), (3, 8.3984375)]}